# SynthTRIPS: A Knowledge-Grounded Framework for Benchmark Query Generation for Personalized Tourism Recommenders

This Colab notebook contains an example of the SynthTRIPS query generation pipeline. The entire pipeline source code is available in our [GitHub Repository](https://github.com/ashmibanerjee/synthTRIPS-code) and the dataset for the knowledge base (KB) can be found on [HuggingFace](https://huggingface.co/datasets/ashmib/synthTRIPS).

More information about the framework and the SynthTRIPS dataset can be found [here](https://ashmibanerjee.github.io/synthTRIPS-website/).

In [ ]:
!pip install openai

In [ ]:
from google.colab import userdata
API_KEY = userdata.get('OPENAI_API_KEY')

from openai import OpenAI
client = OpenAI(api_key=API_KEY)

We use GPT-4o-Mini here, for example purposes. However, other models such as Gemini and LLama (the models used for generation in the paper) can also be used.

In [ ]:
model_id = "gpt-4o-mini"

## Prompt Templates

### Task Instruction - SYS_PROMPT
You are a user query generator for a travel recommender system. You are provided with a list of cities, some information about them, and certain travel filters (for example, regarding the budget or points of interest to visit).
You should include this information in your response to ensure a city travel recommender system suggests the provided cities as an answer to your generated query (response).
You should not include the names of the cities in the response. The query should not exceed 200 characters.

### ICL
Here is an example of a good query for a user who is a {sample_persona}, the provided city information and associated travel filters.
City Information: {sample subgraph}
Travel filters: {sample travel filters}
Generated Query: {sample generated query}

### DK
Here are the following for the new user who is a {persona}:
City Information: {new_subgraph}
Travel filters: {new travel filters}

### Final Task Question
Your task: Guess a query, similar to the example above, that the following person may ask you to generate: {persona}

**Adding the persona in the final task question: inspired from Fig 13: https://arxiv.org/pdf/2406.20094**

In [ ]:
SYS_PROMPT = """
You are a query generator for a travel recommender system for European city trips.
You are provided with a list of cities, some information about them (context), and certain travel filters (for example, regarding the budget or points of interest to visit).
You should include this information in your response to ensure a city travel recommender system suggests the provided cities as an answer to your generated query (response).
You should not include the names of the cities in the response. The query should not exceed 200 characters and should be only relevant for European cities.
"""

In [ ]:
# no persona info, zero shot (baseline)
USER_PROMPT_BASELINE = """

Here are the following for the new user:
City Information: {new_subgraph}
Travel filters: {new_travel_filters}

Your task: Guess a query, that the following person may ask you to generate:
"""

In [ ]:
# persona info, zero shot
USER_PROMPT_ZERO_DK = """
Here are the following for the user who is a {{ persona }}:
City Information: {{ new_subgraph }}
Travel filters: {{ new_travel_filters }}

Your task: Guess a query, that the following person may ask you to generate: {{ persona }}
"""

In [ ]:
# persona info, single-shot
USER_PROMPT_ICL_DK = """
Here is an example of a good query for a user who is a {{ sample_persona }},
the provided city information and associated travel filters.
City Information: {{ sample_subgraph }}
Travel filters: {{ sample_travel_filters }}
Generated Query: {{ sample_generated_query }}

Here are the following for the new user who is a {{ persona }}:
City Information: {{ new_subgraph }}
Travel filters: {{ new_travel_filters }}

Your task: Guess a query, similar to the example above, that the following person may ask you to generate: {{ persona }}
"""

## Example and Test Queries

The example configuration and the test query contain contexts which were retrieved from our KB. In order to replace the test query, you can also use any of the previously generated contexts from the SynthTRIPS dataset, or use the source code to choose new filters and retrieve appropriate context. The example can also be replaced with one of the previously generated queries from the SynthTRIPS dataset.

The options for the travel filters are as follows:

```
{
    "popularity": [
        "low", "medium", "high"
    ],
    "non-sustainable": {
        "budget": [
            "low", "medium", "high"
        ],
        "interests": [
            "Arts & Entertainment", "Outdoors & Recreation" , "Food", "Nightlife Spot", "Shops & Services"
        ],
        "month": [
            "January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"
        ]
    },
    "sustainable": {
        "seasonality": "low",
        "walkability": "great",
        "aqi": "great"
    }
}
```

The sustainable travel filters only have one option each, i.e. the most sustainable option for each filter.

In [1]:
example_config = {
  "config_id": "c_p_0_pop_low_easy",
  "persona": "A veteran travel agent specialized in Asian travel routes.",
  "filters": {
    "popularity": "High",
    "budget": "Low",
    "interests": "Outdoors & Recreation"
  },
  "cities": [
    "Zagreb",
    "Tirana",
    "Tbilisi",
    "Sofia",
    "Samara",
    "Riga",
    "Perm",
    "Moscow",
    "Minsk",
    "Krasnodar",
    "Kazan",
    "Kaliningrad",
    "Izmir",
    "Baku",
    "Ankara"
  ],
  "context": "Zagreb has High popularity and Low budget. In Zagreb you can see Maksimir Park & Zagreb Zoo. In Zagreb you can see Jarun. In Zagreb you can see Ribnjak Park. In Zagreb you can drink at Melin: A cheap bar brimming with young people and students. Offers indoors, a roofed terraced leveled seating and an open terrace facing Tkal\u010di\u0107eva street. Known for crowds of people often drinking even around the bar, in the park and playground in front of it.. In Zagreb you can drink at Portal: Next to Melin, offers same service, the Portal and Melin crowds often mix at the outdoors terraces.. In Zagreb you can do Snow Queen Trophy: Ladies Night Slalom : Zagreb is the only capital in the world that hosts a Ski World Cup race. The slalom event - known as the \"Snow Queen Trophy\" (Snje\u017ena kraljica) - usually takes place in January. It is the most expensive race on the women's world cup tour and an exceptionally popular sporting event in Croatia that attracts a crowd of around 25 thousand people. As of 2007 best skiers in men's competition are also expected to compete on Mount Sljeme that overlooks the city. If you fail to secure your ticket, you can watch the whole event on a giant screen in the Ban Jela\u010di\u0107 Square.. In Zagreb you can drink at Golf: Popular golf-themed cafe/bar.. \nTirana has High popularity and Low budget. In Tirana you can see Persqopi Archeological Site. In Tirana you can do Grand Park along the Artificial Lake: People have picnics and have fun, and the natural beauty allows it because of the huge trees, open spaces, and mountains surrounding them. An artificial lake lies in the middle of the Park. It is popular with locals and tourists as a place to walk around and enjoy a lunch or coffee at one of the restaurants or caf\u00e9s. Over 120 types of plants. A popular swimming pool complex is located nearby.. In Tirana you can do Mount Dajti: 1,612m high (getting to the topmost point is not possible because of a restricted area with antennas and a military zone). Offers great views of Tirana. A very relaxing area, mount Dajti can be reached through an Austrian-built express cable car (one ride takes more than 15 minutes so is worth the cost even if you don't stay atop the mountain). To get to the bottom station of the cable car, take the bus line \"Qender - Linze\" that starts east beside the Skanderbeg place behind the mosque. In the bus tell the conductor \"Teleferik\" and he will show you the station to get out of the bus. From there it is a short 5-minute walk guided by direction signs (the cable car is closed on Tuesdays). In the top station of the cable car, there is a hotel (30\u20ac for a double room [2013]) and a tourist agency offering various activities like horse riding, trekking, excursions, sky sports, ski etc. The area has a limited number of hotels and restaurants, but it is great for taking a breath of fresh air, mountain sports, or to just get out of the city for a while. The '''Pellumba Cave''' in the area is worth a visit. Unfortunately, litter is becoming a problem on Mount Dajti. This detracts from the experience somewhat, but the beautiful scenery still makes the trip worthwhile. Don't forget to visit the spectacular '''Erzeni Canyon''' along the Erzeni River south of the park near the dam.. In Tirana you can do Rally Albania: The event features an international competition in the rough Albanian countryside roads.. In Tirana you can see Ndroqit Castle ruins. In Tirana you can see Preze Castle. \nTbilisi has High popularity and Low budget. In Tbilisi you can do Alexander's Garden: A part of Park April.. In Tbilisi you can see Gudauri. In Tbilisi you can see Telavi. In Tbilisi you can see Ananuri. In Tbilisi you can do Botanical garden: National botanic garden of Georgia, founded 1845. Lying in the Tsavkisis-Tskali Gorge near Narikala Fortres,  is actually more of a large badly maintained park with scenic waterfall than a \"botanical garden\" as there is no scientific signposting available. Very little maintenance is apparent.. In Tbilisi you can do HeidelbergCement TbilisiMarathon: Every year at the end of October a big, international event officially recognized by the International Association of Athletic Federations takes place. There is a half marathon, a 10-km-race and kids' runs. The start and finish is at Rustaveli Avenue. The racecourse guides the runners through this great city to its most interesting sights. The  event is professionally organized by the German company HeidelbergCement Caucasus and a German race director. HeidelbergCement Caucasus is one of the biggest investors in the region. The participation fee goes 100% to charity.. \nSofia has High popularity and Low budget. In Sofia you can do Sofia Bike Tour: Visit the greenest capital of Europe on a bike. Every day Free Bike Tour (12 April\u2013November) at 11AM & 5PM. Escape from the city and join to the Bike Countryside Tour around Sofia. Start Point: National Theater \"Ivan Vazov\". In Sofia you can see Borisova Gradina (Boris Garden). In Sofia you can see Yuzhen Park (South Park). In Sofia you can do Playgrounds: If you're traveling with kids, they might love playgrounds. The \"rich\" one is in the park that holds Vasil Levski stadium (close to the corner with Orlov Most; there are actually two playgrounds close to this corner and you'd look for the one more inside the park).. In Sofia you can see Boyana Church. In Sofia you can drink at Lodkite: In the huge \"Borisova gradina\" park near the stadium (just ask you way around) you'll find this open-air place, located in an old leisure-park parcour (you know, that type of small water-channels where kid can drive around in little gondolas). The boats have gone and the channels serve as improvised sitting places (basic tables made out of wood planks put between the two sides of the channel). There are also some tables on the court and lights decorating the trees illuminate the scene. The public sound system has experimental electronic music, ambiance, progressive rock, or whatever the DJs feel like. The later it gets, the more you'll find people sitting everywhere on the floor on the trees. On warm summer nights, this place is a must-be.. \nSamara has High popularity and Low budget. In Samara you can see Krasnaya Glinka Ski complex. In Samara you can see Viewpoint 'Helipad'. In Samara you can see Dacha Golovkin. \nRiga has High popularity and Low budget. In Riga you can do R\u012bga City Day: Takes place on 14-16 Aug. In Riga you can do Latviabeerfest: Takes place annually at the end of May. The largest international beer festival in the Baltics.. \nPerm has High popularity and Low budget. In Perm you can see Motovilikha open-air museum. In Perm you can see Khohlovka Museum. \nMoscow has High popularity and Low budget. In Moscow you can do Pokrovskie Baths: Has a famous steam room with horseradish-flavored steam. Wide selection of brooms from birch tree, tatarian maple, lime tree, and eucalyptus. Prices are cheaper than other banyas.. In Moscow you can do Rzehvskye Baths: 120 years old but recently renovated.. In Moscow you can do Kva-Kva Water Park: Water Park affiliated with Maxima Hotels (discounts for guests). There are 7 high trills (90-120 meters length) and a pleasant surprise for extremers \u2013 Tsunami trill \u2013 unique in Russia. There are also 4-line trills \u2013 Multislide and a special area for kids \u2013 a small tropical town with shallow pool. - Kva-Kva Lagoone offers hydromassage. There\u2019s also pure Russian bath, Finnish sauna, Turkish bath (hamam) and Kva-Kva SPA-salon. - Night discos take place every weekend, with free admission for Maxima Hotel guests.. \nMinsk has High popularity and Low budget. In Minsk you can do Skiing resorts: These are the most popular places to have a rest. They provide wide range of winter activities: skiing, snowboarding, skating, tubing etc.. In Minsk you can see Ozertso - Open Air Museum of Rural Architecture and Life. In Minsk you can do Children's Railroad: This is a 750 mm (2 ft 5 1\u20442 in) narrow gauge railroad loop passing through the Chelyuskinites Park. The railway line counts 3 stations (listed from north to south): Zaslonovo, Pionerskaya (or Pionerskaja) and Sosnovy Bor (or Sosnovyj Bor); and a pair of rail loops after the end stops.. In Minsk you can do Minsk Sea: This  is an artificial reservoir.  There's a free public beach, and pedal-boat and catamaran rental.. In Minsk you can see Khatyn Memorial Complex. In Minsk you can see Mound of Glory. \nKrasnodar has High popularity and Low budget. In Krasnodar you can see Museum of Military Technologies Oruzhie Pobedy. \nKazan has High popularity and Low budget. In Kazan you can do Victory Park: The park includes a memorial to the heroes of the Great Patriotic War. The Victory Column is 42m high.. In Kazan you can see Suyumbike Tower. In Kazan you can see Kazan Kremlin. In Kazan you can see Kul-Sharif Mosque. \nKaliningrad has High popularity and Low budget. In Kaliningrad you can do Trek in the forests: Soak up nature in the untouched forests. In Kaliningrad you can see Fishing Village. In Kaliningrad you can see The old cathedral of K\u00f6nigsberg. \nIzmir has High popularity and Low budget. In Izmir you can see \u0130zmir Agora Open Air Museum. In Izmir you can see Konak Square. In Izmir you can see Kadifekale. \nBaku has High popularity and Low budget. In Baku you can see Oil Rocks. In Baku you can do Naftalan Bath: Oil baths are famous for Naftalan, a city close to Ganca. However you can also get an oil treatment in Baku, with Naftalan oil imported from there. It's like a small hospital and it's not a recreactional thing to do usually. However it's great fun and positively strange to dip for 15min into hot crude oil. Choose yourself whether this is a save and healthy thing to do.. In Baku you can see Yanar Dagh. In Baku you can see Hajji Gayyib bathhouse. In Baku you can do Boat tours of Baku: Offered from the Baku Blvd. Lasts 40 minutes.. In Baku you can do AVIS: AVIS Rent a Car provides cars, SUV and 4WD. Airport pick up and delivery service is also available.. \nAnkara has High popularity and Low budget. In Ankara you can do Goksu Park: Enjoy the scenic Susuz Lake (G\u00f6lu). In Ankara you can do Atat\u00fcrk Forest Farm and ZOO: This is an expansive recreational farming area, which houses a zoo, several small agricultural farms, greenhouses, restaurants, a dairy farm and a brewery. It was started by Kemal Atat\u00fcrk, who wanted to prove that the neglected steppes around Ankara could be converted into a lush, fertile agricultural land.. In Ankara you can do Guvenpark: A small park surrounding the Security Monument. In Ankara you can see Arslanhane Mosque. In Ankara you can see Roman Bath. In Ankara you can see Roman Road of Ankara. \n",

  "query": "Which European cities are popular and budget-friendly with outdoor recreational activities and parks to explore?"
}

In [ ]:
# Test query

config_3 = {
    "config_id": "c_easy_2",
    "id": "p_76",
    "persona": "A travel blogger known for their captivating storytelling and stunning photography, attracting thousands of followers to their blog and social media channels",
    "kg_filters": {
        'popularity': 'high',
        'budget': 'medium',
        'interests': 'Arts & Entertainment'
        },
    "cities": [
        "Vilnius",
        "Tallinn",
        "Sarajevo",
        "Nicosia",
        "Budapest",
        "Bratislava",
        "Belgrade"
    ],
    "context": [
        "Vilnius has high popularity.\nVilnius has medium budget.\nIn Vilnius you can do Lithuanian Opera and Ballet Theatre: beautiful ornate building. Three or four weekly performances..\nIn Vilnius you can do Lithuanian Russian Drama Theatre: performances mostly done in Russian language..\nIn Vilnius you can eat at Balti Drambliai (White Elephants): Cozy vegetarian-vegan cafe, world/reggae music, live concerts in the evenings..\nIn Vilnius you can do Oskaras Kor\u0161unovas Theatre (OKT): one of the most original modern theatres in Europe.\nIn Vilnius you can drink at Artistai: The large popular courtyard is a great place to relax and watch the sunset. Friendly service. Rock music..\nIn Vilnius you can see Angel of Uzupis Statue.\nIn Vilnius you can drink at Lindyhop.lt Klubas: The coolest and oldest Lindy Hop club in Lithuania..\nIn Vilnius you can drink at Prie Universiteto: Rock/punk rock music, friendly stuff and cheap prices. Best place for students and other not so serious people..\nIn Vilnius you can see St. Peter's and Paul's Church.\nIn Vilnius you can eat at \u010cagino (Chagin's): Named after the famous Russian architect Nikolai Chagin. Live Russian music played on a guitar..\nIn Vilnius you can see St. Ann's Church.\n",
        "Tallinn has high popularity.\nTallinn has medium budget.\nIn Tallinn you can do Birgitta Festival: Music and theatre festival, held at the ruins of the historical Pirita (St. Bridget's) convent..\nIn Tallinn you can drink at Von Krahli: An avant-garde theatre/bar..\nIn Tallinn you can see Song Festival Grounds.\nIn Tallinn you can do Saku Suurhall: Rocca al Mare. Estonia's largest concert and exhibition space, the venue for the 2002 Eurovision Song Contest. The hall and its facilities include an excellent shopping centre that can easily be reached by trolleybus 6..\nIn Tallinn you can see Town Hall.\nIn Tallinn you can drink at Parlament: Bubblegum pop and live events..\nIn Tallinn you can do Tallinn Music Week: Showcase festival, aiming to stage the best and most outstanding Estonian talent on two nights in Tallinn's most vibrant live venues, as well as a networking event for the music industry professionals..\nIn Tallinn you can drink at Teater: Basement club on Vabaduse square. Popular with late twens and smaller groups. Music focus is on 90s and 00s. Door policy is relaxed but most people are dressed up. Usually starts going at midnight..\nIn Tallinn you can see St. Nicholas church.\n",
        "Sarajevo has high popularity.\nSarajevo has medium budget.\nIn Sarajevo you can do Sarajevo Jazz Festival: A very eclectic festival, and the largest of its kind in the Balkans..\nIn Sarajevo you can drink at Opera Bar/Caf\u00e9: Fast WiFi connection, but the waitstaff are often unfriendly and inattentive. It attracts the acting and musical community among the regulars, though this isn't an exclusive kind of place. A bit smoky..\nIn Sarajevo you can see Sarajevo Art Gallery.\nIn Sarajevo you can do Sarajevo Film Festival: One of the best film festivals in Europe and the largest of its kind in Southeastern Europe..\nIn Sarajevo you can see Svrzina kuca (Svrzo house).\nIn Sarajevo you can drink at Central Cafe: Cocktail bar with great music.  Get there early or call to reserve a table.  The place is very busy until midnight when people leave to hit the various nightclubs around town. The street is a whole promenade with many other cafes around..\nIn Sarajevo you can see St. Anthony\u00b4s Church.\n",
        "Nicosia has high popularity.\nNicosia has medium budget.\nIn Nicosia you can see Famagusta Gate.\nIn Nicosia you can see The Nicosia Municipal Theatre.\nIn Nicosia you can do Ifantourgio: 67-71 Lefkonos Str., Phaneromeni, 1011 Old Nicosia. The name of this place translates as ''The Weaving Mill'' and is in fact a converted old factory. Very arty, no blockbusters here. You can sip your wine as you watch an alternative movie..\nIn Nicosia you can see Byzantine Museum.\nIn Nicosia you can drink at The Kush Bar: Eleftheria Square, 1521 Nicosia. Opened in 2006, decorated with ultramodern minimalistic furniture and overlooking the dry moat surrounding Nicosia's venetian walls..\nIn Nicosia you can drink at Scorpios Club: Probably the only ''discoth\u00e8que'' in Nicosia having survived the test of time. Opened in the early 1970s and has reinvented itself (after several facelifts) since..\nIn Nicosia you can drink at Svoura Caf\u00e9-Bar: 5 Thermopylon, Old town. Serves coffee, snacks and alcohol; dance all night long. Free Wi-fi..\n",
        "Budapest has high popularity.\nBudapest has medium budget.\nIn Budapest you can do Budapest Spring Festival: A dazzling variety of cultural events mainly revolving around classical music and performing arts - including folklore..\nIn Budapest you can do Firework above Danube River: National Holiday.  Sound & Light Show with firework. Around half hour open air program..\nIn Budapest you can do Jewish Summer Festival: Another array of cultural and music events, with a Jewish touch..\n",
        "Bratislava has high popularity.\nBratislava has medium budget.\nIn Bratislava you can see St. Clare's Church.\nIn Bratislava you can see Slovak National Theatre.\nIn Bratislava you can see Bratislava Castle.\nIn Bratislava you can drink at BeAbout: Music club near the riverside popular with younger crowd. Good selection of beers including Belgian specialty beers..\nIn Bratislava you can drink at UFO: If you want to go for extraordinary, visit UFO. It looks like a clich\u00e9 flying saucer and it's a high on a New Bridge (the member of great-towers.com), and you have to use a lift to go there. There is a good restaurant and lounge bar and observation deck. The view is incomparable..\n",
        "Belgrade has high popularity.\nBelgrade has medium budget.\nIn Belgrade you can drink at Inex Film: a squat on the North side of the center, on the shore of the Danube, with weekly film screenings, art exhibitions, and DJ parties..\nIn Belgrade you can see The Residence of Princess Ljubica.\nIn Belgrade you can see Terazije and Kralja Milana streets.\nIn Belgrade you can see Church Ru\u017eica.\n"
    ]
}

## Test Cases: Vanilla (V), Personalized Zero-Shot (P0) and Personalized Single-Shot (P1)

In [ ]:
# Case 1 (V): Non personalized Zero shot: Baseline
updated_user_prompt = USER_PROMPT_BASELINE.format(new_subgraph=config_3["context"], new_travel_filters=config_3['filters'])
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
         {"role": "system", "content": SYS_PROMPT},
         {"role": "user", "content":updated_user_prompt}
         ],
    stream=False,
)
print('Supplied sub-graph:', config_3["context"])
print('Supplied travel filters:', config_3["filters"])
print(f'\n \n Generated query: {response.choices[0].message.content}')

Supplied sub-graph: ["Vilnius has high popularity.\nVilnius has medium budget.\nIn Vilnius you can do Lithuanian Opera and Ballet Theatre: beautiful ornate building. Three or four weekly performances..\nIn Vilnius you can do Lithuanian Russian Drama Theatre: performances mostly done in Russian language..\nIn Vilnius you can eat at Balti Drambliai (White Elephants): Cozy vegetarian-vegan cafe, world/reggae music, live concerts in the evenings..\nIn Vilnius you can do Oskaras Koršunovas Theatre (OKT): one of the most original modern theatres in Europe.\nIn Vilnius you can drink at Artistai: The large popular courtyard is a great place to relax and watch the sunset. Friendly service. Rock music..\nIn Vilnius you can see Angel of Uzupis Statue.\nIn Vilnius you can drink at Lindyhop.lt Klubas: The coolest and oldest Lindy Hop club in Lithuania..\nIn Vilnius you can drink at Prie Universiteto: Rock/punk rock music, friendly stuff and cheap prices. Best place for students and other not so ser

In [ ]:
# Case 2 (P0): Personalized but without ICL (Zero-Shot)

updated_user_prompt = USER_PROMPT_ZERO_DK.format(persona=config_3["persona"], new_subgraph=config_3["context"], new_travel_filters=config_3['filters'])
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
         {"role": "system", "content": SYS_PROMPT},
         {"role": "user", "content":updated_user_prompt}
         ],
    stream=False,
)

print("Persona:", config_3["persona"])
print('Supplied sub-graph:', config_3["context"])
print('Supplied travel filters:', config_3["filters"])
print(f'\n \n Generated query: {response.choices[0].message.content}')

Persona: A travel blogger known for their captivating storytelling and stunning photography, attracting thousands of followers to their blog and social media channels
Supplied sub-graph: ["Vilnius has high popularity.\nVilnius has medium budget.\nIn Vilnius you can do Lithuanian Opera and Ballet Theatre: beautiful ornate building. Three or four weekly performances..\nIn Vilnius you can do Lithuanian Russian Drama Theatre: performances mostly done in Russian language..\nIn Vilnius you can eat at Balti Drambliai (White Elephants): Cozy vegetarian-vegan cafe, world/reggae music, live concerts in the evenings..\nIn Vilnius you can do Oskaras Koršunovas Theatre (OKT): one of the most original modern theatres in Europe.\nIn Vilnius you can drink at Artistai: The large popular courtyard is a great place to relax and watch the sunset. Friendly service. Rock music..\nIn Vilnius you can see Angel of Uzupis Statue.\nIn Vilnius you can drink at Lindyhop.lt Klubas: The coolest and oldest Lindy Hop 

In [ ]:
# Case 3 (P1): Personalized with ICL (persona in user prompt) (Single-Shot)

updated_user_prompt = USER_PROMPT_ICL_DK.format(sample_persona=example_config["persona"], sample_subgraph=example_config["context"], sample_travel_filters=example_config['filters'],
                                                # sample_generated_query="Suggest cities in March with low-season rates that offer delicious food and medium budget options for dining and accommodations.",
                                                sample_generated_query= example_config["query"],
                                                persona=config_3["persona"], new_subgraph=config_3["context"], new_travel_filters=config_3['filters'])
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
         {"role": "system", "content": SYS_PROMPT},
         {"role": "user", "content":updated_user_prompt}
         ],
    stream=False,
)
print("Persona:", config_3["persona"])
print('Supplied sub-graph:', config_3["context"])
print('Supplied travel filters:', config_3["filters"])
print(f'\n \n Generated query: {response.choices[0].message.content}')

Persona: A travel blogger known for their captivating storytelling and stunning photography, attracting thousands of followers to their blog and social media channels
Supplied sub-graph: ["Vilnius has high popularity.\nVilnius has medium budget.\nIn Vilnius you can do Lithuanian Opera and Ballet Theatre: beautiful ornate building. Three or four weekly performances..\nIn Vilnius you can do Lithuanian Russian Drama Theatre: performances mostly done in Russian language..\nIn Vilnius you can eat at Balti Drambliai (White Elephants): Cozy vegetarian-vegan cafe, world/reggae music, live concerts in the evenings..\nIn Vilnius you can do Oskaras Koršunovas Theatre (OKT): one of the most original modern theatres in Europe.\nIn Vilnius you can drink at Artistai: The large popular courtyard is a great place to relax and watch the sunset. Friendly service. Rock music..\nIn Vilnius you can see Angel of Uzupis Statue.\nIn Vilnius you can drink at Lindyhop.lt Klubas: The coolest and oldest Lindy Hop 